In [28]:
%cd /teamspace/studios/this_studio/2024/07/flood_prediction

/teamspace/studios/this_studio/2024/07/flood_prediction


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [29]:
import pandas as pd
import numpy as np

In [30]:
train_df = pd.read_csv('train.csv')

In [31]:
train_df.head()

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,0,5,8,5,8,6,4,4,3,3,...,5,3,3,5,4,7,5,7,3,0.445
1,1,6,7,4,4,8,8,3,5,4,...,7,2,0,3,5,3,3,4,3,0.450
2,2,6,5,6,7,3,7,1,5,4,...,7,3,7,5,6,8,2,3,3,0.530
3,3,3,4,6,5,4,8,4,7,6,...,2,4,7,4,4,6,5,7,5,0.535
4,4,5,3,2,6,4,4,3,3,3,...,2,2,6,6,4,1,2,3,5,0.415


In [32]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1117957 entries, 0 to 1117956
Data columns (total 22 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   id                               1117957 non-null  int64  
 1   MonsoonIntensity                 1117957 non-null  int64  
 2   TopographyDrainage               1117957 non-null  int64  
 3   RiverManagement                  1117957 non-null  int64  
 4   Deforestation                    1117957 non-null  int64  
 5   Urbanization                     1117957 non-null  int64  
 6   ClimateChange                    1117957 non-null  int64  
 7   DamsQuality                      1117957 non-null  int64  
 8   Siltation                        1117957 non-null  int64  
 9   AgriculturalPractices            1117957 non-null  int64  
 10  Encroachments                    1117957 non-null  int64  
 11  IneffectiveDisasterPreparedness  1117957 non-null 

In [43]:
k = train_df.sample(frac=1)[:10000]
k.to_csv('small_train.csv',index=False)

In [16]:
train_df.isna().sum()

id                                 0
MonsoonIntensity                   0
TopographyDrainage                 0
RiverManagement                    0
Deforestation                      0
Urbanization                       0
ClimateChange                      0
DamsQuality                        0
Siltation                          0
AgriculturalPractices              0
Encroachments                      0
IneffectiveDisasterPreparedness    0
DrainageSystems                    0
CoastalVulnerability               0
Landslides                         0
Watersheds                         0
DeterioratingInfrastructure        0
PopulationScore                    0
WetlandLoss                        0
InadequatePlanning                 0
PoliticalFactors                   0
FloodProbability                   0
dtype: int64

In [17]:
train_df = train_df.drop(columns='id',axis=1)

In [18]:
from sklearn.preprocessing import MinMaxScaler

In [19]:
scaler = MinMaxScaler()

In [22]:
X_train, y_train = train_df.iloc[:, :-1], train_df.iloc[:, -1]

In [23]:
X_train

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors
0,5,8,5,8,6,4,4,3,3,4,2,5,3,3,5,4,7,5,7,3
1,6,7,4,4,8,8,3,5,4,6,9,7,2,0,3,5,3,3,4,3
2,6,5,6,7,3,7,1,5,4,5,6,7,3,7,5,6,8,2,3,3
3,3,4,6,5,4,8,4,7,6,8,5,2,4,7,4,4,6,5,7,5
4,5,3,2,6,4,4,3,3,3,3,5,2,2,6,6,4,1,2,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117952,3,3,4,10,4,5,5,7,10,4,3,7,8,7,2,2,1,4,6,4
1117953,2,2,4,3,9,5,8,1,3,5,5,9,4,4,3,7,4,9,4,5
1117954,7,3,9,4,6,5,9,1,3,4,5,5,5,5,5,6,5,5,2,4
1117955,7,3,3,7,5,2,3,4,6,4,6,6,8,5,3,4,6,7,6,4


In [21]:
pd.DataFrame(scaler.fit_transform(train_df),columns = train_df.columns)

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,0.3125,0.444444,0.3125,0.470588,0.352941,0.235294,0.2500,0.1875,0.1875,0.222222,...,0.294118,0.176471,0.1875,0.3125,0.235294,0.388889,0.263158,0.4375,0.1875,0.363636
1,0.3750,0.388889,0.2500,0.235294,0.470588,0.470588,0.1875,0.3125,0.2500,0.333333,...,0.411765,0.117647,0.0000,0.1875,0.294118,0.166667,0.157895,0.2500,0.1875,0.375000
2,0.3750,0.277778,0.3750,0.411765,0.176471,0.411765,0.0625,0.3125,0.2500,0.277778,...,0.411765,0.176471,0.4375,0.3125,0.352941,0.444444,0.105263,0.1875,0.1875,0.556818
3,0.1875,0.222222,0.3750,0.294118,0.235294,0.470588,0.2500,0.4375,0.3750,0.444444,...,0.117647,0.235294,0.4375,0.2500,0.235294,0.333333,0.263158,0.4375,0.3125,0.568182
4,0.3125,0.166667,0.1250,0.352941,0.235294,0.235294,0.1875,0.1875,0.1875,0.166667,...,0.117647,0.117647,0.3750,0.3750,0.235294,0.055556,0.105263,0.1875,0.3125,0.295455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117952,0.1875,0.166667,0.2500,0.588235,0.235294,0.294118,0.3125,0.4375,0.6250,0.222222,...,0.411765,0.470588,0.4375,0.1250,0.117647,0.055556,0.210526,0.3750,0.2500,0.477273
1117953,0.1250,0.111111,0.2500,0.176471,0.529412,0.294118,0.5000,0.0625,0.1875,0.277778,...,0.529412,0.235294,0.2500,0.1875,0.411765,0.222222,0.473684,0.2500,0.3125,0.443182
1117954,0.4375,0.166667,0.5625,0.235294,0.352941,0.294118,0.5625,0.0625,0.1875,0.222222,...,0.294118,0.294118,0.3125,0.3125,0.352941,0.277778,0.263158,0.1250,0.2500,0.454545
1117955,0.4375,0.166667,0.1875,0.411765,0.294118,0.117647,0.1875,0.2500,0.3750,0.222222,...,0.352941,0.470588,0.3125,0.1875,0.235294,0.333333,0.368421,0.3750,0.2500,0.477273


In [24]:
from sklearn.model_selection import train_test_split